In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
!apt-get update
!apt install chromium-chromedriver
!pip install selenium
!pip install bs4

In [0]:
import os
import time
import sys
import requests
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
from collections import OrderedDict 
chromedriver = "/usr/bin/chromedriver"
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
d = webdriver.Chrome(chromedriver,options=chrome_options)
destinationDir = "/content/drive/Shared drives/PankajCCSF/test"
url = 'https://edu.tuts.workers.dev/Udacity%20-%20Collections%20[300%20GB]/Nanodegrees/Business%20Analytics%20Nanodegree%20v2.0.0/'
d.get(url)
time.sleep(7)
page_html = d.page_source

#crawler starts here
def crawler(page_html):
    folderDic = {}
    fileDic = {}
    soup = BeautifulSoup(page_html, 'html.parser')
    for fileFolders in soup.select('li.mdui-list-item a'):
        if (fileFolders.contents[1].text.find("folder_open") > -1):
            tempFolderName = fileFolders.contents[1].text
            folderName = tempFolderName.replace("folder_open", "").strip().strip().replace("/","")
            folderDic[folderName] = "https://edu.tuts.workers.dev" + fileFolders['href']
        else:
            tempFileName = fileFolders.contents[1].text
            fileName = tempFileName.replace("insert_drive_file", "").strip().replace("/","")
            fileDic[fileName] = "https://edu.tuts.workers.dev" + fileFolders['href'].replace("?a=view", "")

    return folderDic,fileDic
    #crawler ends here

folderDictonary,fileDictonary = crawler(page_html)
for folderName, folderUrl in folderDictonary.items():
  saveDir = os.path.join(destinationDir,folderName)
  if(os.path.exists(saveDir)):
    print("skipping => " + saveDir)
    continue
  print("Downloading =>" + saveDir)  
  try:
    os.mkdir(saveDir)
  except Exception as e:
    pass  
  d.get(folderUrl)
  time.sleep(9)
  lvlTwoSource = d.page_source
  folderDictonaryLvlTwo,fileDictonaryLvlTwo = crawler(lvlTwoSource)
  if(len(folderDictonaryLvlTwo)==0 and len(fileDictonaryLvlTwo)==0):
    print(saveDir)
    print("Content might not have been downloaded for above folder . Please consider increasing sleep time on line number 50 and delete the empty folder")
  # sys.exit()
  for folderNameTwo, folderUrlTwo in folderDictonaryLvlTwo.items(): 
    lvlTwoSaveDir = os.path.join(saveDir,folderNameTwo)
    try:
      os.mkdir(lvlTwoSaveDir)
    except Exception as e:
      pass

  #actual file processing

  for fileNameTwo, fileUrlTwo in fileDictonaryLvlTwo.items(): 
    if(os.path.exists(os.path.join(saveDir,fileNameTwo))):
      print("skipping => " + os.path.join(saveDir,fileNameTwo))
      continue 
    r = requests.get(fileUrlTwo,stream=True)
    with open(os.path.join(saveDir,fileNameTwo),"ab+") as f:
      f.write(r.content)

  # print(folderName + "=>" + folderUrl)

for fileNameMainDir, fileUrlMain in fileDictonary.items():
  # if(os.path.exists(os.path.join(destinationDir,fileNameMainDir))):
  #   print("skipping =>" + os.path.join(destinationDir,fileNameMainDir))  
  #   continue
  r = requests.get(fileUrlMain,stream=True)
  with open(os.path.join(destinationDir,fileNameMainDir),"ab+") as f:
    print("Downloading =>" + os.path.join(destinationDir,fileNameMainDir))
    f.write(r.content)
